In [50]:

from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

In [51]:
# Каталог с данными для обучения
train_dir = 'cadmnist/train'
# Каталог с данными для проверки
val_dir = 'cadmnist/val'
# Каталог с данными для тестирования
test_dir = 'cadmnist/test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 1)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 464
# Количество изображений для проверки
nb_validation_samples = 149
# Количество изображений для тестирования
# nb_test_samples = 86
# количество классов
num_classes=5

In [52]:
model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(num_classes, activation='softmax'))

model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dense(5))
model.add(layers.Activation('softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 148, 148, 32)      320       
                                                                 
 activation_30 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 activation_31 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                 

In [53]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical')

# test_generator = datagen.flow_from_directory(
#     test_dir,
#     target_size=(img_width, img_height),
#     color_mode='grayscale',
#     batch_size=batch_size,
#     class_mode='categorical')

Found 464 images belonging to 5 classes.
Found 149 images belonging to 5 classes.


In [54]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [55]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
58/58 [==============================] - 9s 141ms/step - loss: 1.1871 - accuracy: 0.5172 - val_loss: 0.7400 - val_accuracy: 0.7083
Epoch 2/10
58/58 [==============================] - 8s 137ms/step - loss: 0.6303 - accuracy: 0.7888 - val_loss: 0.5598 - val_accuracy: 0.8264
Epoch 3/10
58/58 [==============================] - 8s 135ms/step - loss: 0.3320 - accuracy: 0.8793 - val_loss: 0.4258 - val_accuracy: 0.8681
Epoch 4/10
58/58 [==============================] - 8s 134ms/step - loss: 0.1942 - accuracy: 0.9267 - val_loss: 0.4807 - val_accuracy: 0.8889
Epoch 5/10
58/58 [==============================] - 9s 155ms/step - loss: 0.1304 - accuracy: 0.9504 - val_loss: 0.4703 - val_accuracy: 0.8750
Epoch 6/10
58/58 [==============================] - 8s 146ms/step - loss: 0.0919 - accuracy: 0.9526 - val_loss: 0.4762 - val_accuracy: 0.8889
Epoch 7/10
58/58 [==============================] - 8s 145ms/step - loss: 0.1174 - accuracy: 0.9720 - val_loss: 0.4650 - val_accuracy: 0.8472
Epoch 

In [56]:
# scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
# print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

In [57]:
model.save('cadmnist.h5')

In [58]:
model = models.load_model('cadmnist.h5')
classes=["bearing", "bolt", "nut", "seal", "washer"]

img=cv2.imread('cadmnist/test/bearing.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/test/bolt.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/test/nut.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/test/seal.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/test/washer.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

1/1 [==============================] - 0s 90ms/step
[9.99855280e-01 4.06049695e-17 1.18049975e-07 1.44557271e-04
 2.49730014e-09]
bearing 99%
1/1 [==============================] - 0s 30ms/step
[1.3737770e-06 9.9765009e-01 4.6800847e-05 3.9256788e-06 2.2977865e-03]
bolt 99%
1/1 [==============================] - 0s 27ms/step
[9.7168484e-10 6.1579096e-11 9.9999964e-01 7.4730072e-10 3.0429376e-07]
nut 99%
1/1 [==============================] - 0s 26ms/step
[6.7240368e-03 1.3847475e-11 3.2365024e-03 9.9002129e-01 1.8220593e-05]
seal 99%
1/1 [==============================] - 0s 26ms/step
[2.0917631e-05 5.6327961e-05 4.0734536e-05 8.5439287e-06 9.9987352e-01]
washer 99%
